In [ ]:
import base64
import requests

import pandas as pd
from glob import glob
import time
import pickle

In [ ]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
model = "gpt-4-vision-preview"

max_tokens = 1000
detail = "high"

path_openai_API_key = r"path/to/your/API/key" #For Example: 'C:\Users\Ivan\Documents\Test-Key-GPT.txt'
ddbb = r'DDBB/to/evaluate/path/.csv' # For Example: r'comparisons\agedb_comparison_1000_balanced.csv'

In [ ]:
with open(path_openai_API_key, 'r') as file:
    api_key = file.read().replace('\n', '') # Change directly for the api key if you don't want to use a file

In [ ]:
prompt_face_recog = "I will give you two images. I want you to tell me if they are from the same identity or not. Answer with yes or no only. Give me a number between 0 and 100 representing how sure you are of your response. Try not to give exactly 0 or 100."

In [ ]:
prompt_explainability = "I will give you two images. I want you to tell me if they are from the same identity or not. Answer with yes or no only. Give me a number between 0 and 100 representing how sure you are of your response. Try not to give exactly 0 or 100. Tell me why you answer that."

In [ ]:
prompt_soft_bio = "I will give you two images. I want you to tell me if they are from the same identity or not. Answer with yes or no only. Give me a number between 0 and 100 representing how sure you are of your response. Separate this number from the response with a line break. Try not to give exactly 0 or 100. Also for the next list of attributes, I want you to give me two lists representing whether the images in the cell have that attribute or not. Answer to me in a python-like list, containing the attributes that are present. The array of the left image is called left_array and the one from the right image is called righ_array. The list of attributes is: ['Male', 'Young', 'Middle_Aged', 'Senior','Asian', 'White', 'Black', 'Indian', 'Rosy_Cheeks', 'Shiny_Skin', 'Bald','Wavy_Hair', 'Receding_Hairline', 'Bangs', 'Sideburns', 'Black_Hair','Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'No_Beard', 'Mustache','5_o_Clock_Shadow', 'Goatee', 'Oval_Face', 'Square_Face', 'Round_Face','Double_Chin', 'High_Cheekbones', 'Chubby', 'Obstructed_Forehead','Fully_Visible_Forehead', 'Brown_Eyes', 'Bags_Under_Eyes','Bushy_Eyebrows', 'Arched_Eyebrows', 'Mouth_Closed', 'Smiling','Big_Lips', 'Big_Nose', 'Pointy_Nose', 'Heavy_Makeup', 'Wearing_Hat','Wearing_Earrings', 'Wearing_Necktie', 'Wearing_Lipstick', 'No_Eyewear','Eyeglasses', 'Attractive']"

In [ ]:
df = pd.read_csv(ddbb)
print(df.shape)
df.head()

In [ ]:
responses = [] 
prompt = prompt_explainability # Change the prompt depending on the task
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}
for img in df['combined_img']:
    print(img)
    base64_image = encode_image(img)

    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": prompt},
            {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                        "detail": detail
                    }
                }
            ]
            },
        ],
        "max_tokens": max_tokens
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload).json()
    print(response)
    if 'error' in response.keys():
        if response['error']['message'] != 'Your input image may contain content that is not allowed by our safety system.':
            print('Error')
            break
        else:
            print('Error but continue')

    responses.append(response)
    print('Waiting 5 second...')
    time.sleep(5) # Added time to avoid the API to block us

dict_responses = {'question': prompt, 'responses': responses}
# We recomend to save the responses in a pickle file
# pickle.dump(dict_responses, open(r'path/to/save/responses', 'wb'))

In [ ]:
dict_responses